In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
from typing import Tuple

# ===== 1. Исходные данные и параметры =====
input_files = {
    "combined_scores": r"../../../depmap_datasets/prepared_datasets/depscores.csv",
    "merged_data": r"../../../depmap_datasets/prepared_datasets/cancer_cell_lines_merged_data.csv",
    "lfc_data": r"../../../depmap_datasets/prepared_datasets/lfc_pim-447_prism.csv"
}

output_files = {
    "correlation_results": r"./correlation_results_pim-447_lfc_vs_depscore_by_cancer_type.csv",
    "p_value_results": r"./p_value_results_pim-447_lfc_vs_depscore_by_cancer_type.csv"
}

genes_to_keep = [
    "ABL1", "ABL2", "AXL", "BMX", "BTK", "CAMK1D", "CAMK2G", "CAV1", "CSK", 
    "EGFR", "EPHB4", "FER", "FES", "FGR", "FRK", "FYN", "HCK", "LCK", "LYN", "MAP4K2", 
    "MAP4K5", "MINK1", "PTK2", "PTK2B", "SIK1", "SIK2", "SIK3", "SRC", "STK10", 
    "STK24", "STK4", "SYK", "TBK1", "TEC", "TNK2", "YES1"
]

excluded_cancers = ["Engineered"]

# ===== 2. Основной обработчик данных (с удалением дубликатов LFC и выводом информации) =====
def process_data() -> pd.DataFrame:
    """Загружает и объединяет данные, удаляя дубликаты LFC и фильтруя по ≥10 линиям."""
    # Загрузка данных
    df_combined = pd.read_csv(input_files["combined_scores"])
    df_merged = pd.read_csv(input_files["merged_data"])
    df_lfc = pd.read_csv(input_files["lfc_data"])
    
    # Удаление дубликатов LFC (оставляем строку с наименьшим LFC)
    initial_count = len(df_lfc)
    duplicates_count = df_lfc.duplicated(subset='ModelID', keep=False).sum()
    
    df_lfc = df_lfc.sort_values('LFC').drop_duplicates('ModelID', keep='first')
    removed_count = initial_count - len(df_lfc)
    
    print(f"\nОбработка LFC данных:")
    print(f" - Начальное количество строк: {initial_count}")
    print(f" - Найдено дубликатов ModelID: {duplicates_count}")
    print(f" - Удалено строк: {removed_count}")
    print(f" - Осталось уникальных ModelID: {len(df_lfc)}")
    
    # Фильтрация генов
    genes_in_file = [gene for gene in genes_to_keep if gene in df_combined.columns]
    df_filtered = df_combined[['DepMap_ID'] + genes_in_file]
    
    # Объединение данных
    df_with_disease = pd.merge(
        df_filtered,
        df_merged[['ModelID', 'primary_disease_renamed']],
        left_on='DepMap_ID',
        right_on='ModelID',
        how='inner'
    ).drop(columns=['ModelID'])
    
    # Добавление LFC
    df_final = pd.merge(
        df_with_disease,
        df_lfc[['ModelID', 'LFC']],
        left_on='DepMap_ID',
        right_on='ModelID',
        how='inner'
    ).drop(columns=['ModelID'])
    
    # Фильтрация по заболеваниям (≥10 линий и исключение указанных типов)
    disease_counts = df_final['primary_disease_renamed'].value_counts()
    valid_diseases = disease_counts[disease_counts >= 10].index
    
    df_final = df_final[
        df_final['primary_disease_renamed'].isin(valid_diseases) & 
        ~df_final['primary_disease_renamed'].isin(excluded_cancers)
    ]
    
    print(f"\nФильтрация по типам рака:")
    print(f" - Общее число линий после фильтрации: {len(df_final)}")
    print(f" - Учтённые типы рака ({len(valid_diseases)}):")
    print(df_final['primary_disease_renamed'].value_counts().to_string())
    
    return df_final

# ===== 3. Анализ корреляций (с фильтрацией по ≥10 точкам) =====
def analyze_correlations(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Считает корреляции Спирмена, требуя ≥10 точек. Иначе corr=0, pval=1."""
    results = {'correlations': [], 'p_values': []}
    cancer_types = data['primary_disease_renamed'].unique()
    
    for cancer_type in cancer_types:
        subset = data[data['primary_disease_renamed'] == cancer_type]
        genes_data = subset.drop(columns=['DepMap_ID', 'primary_disease_renamed', 'LFC'])
        lfc_data = subset['LFC']
        
        # Очистка от inf/NaN
        genes_data = genes_data.replace([np.inf, -np.inf], np.nan)
        lfc_data = lfc_data.replace([np.inf, -np.inf], np.nan)
        
        corr_results = {'Cancer_Type': cancer_type}
        pval_results = {'Cancer_Type': cancer_type}
        
        for gene in genes_data.columns:
            # Объединяем данные гена и LFC, удаляя строки с NaN
            combined = pd.DataFrame({
                'gene': genes_data[gene],
                'lfc': lfc_data
            }).dropna()
            
            n_points = len(combined)
            if n_points >= 10:
                corr, pval = stats.spearmanr(combined['gene'], combined['lfc'])
                corr_results[gene] = corr
                pval_results[gene] = pval
            else:
                corr_results[gene] = 0.0
                pval_results[gene] = 1.0
                print(f"Предупреждение: Для {cancer_type}->{gene} только {n_points} точек (требуется ≥10)")
        
        results['correlations'].append(corr_results)
        results['p_values'].append(pval_results)
    
    return pd.DataFrame(results['correlations']), pd.DataFrame(results['p_values'])

# ===== 4. Главная функция =====
def main():
    print("Загрузка и обработка данных...")
    df = process_data()
    
    if df.empty:
        raise ValueError("Данные после фильтрации пусты! Проверьте входные файлы и параметры.")
    
    print("\nАнализ корреляций...")
    corr_df, pval_df = analyze_correlations(df)
    
    print("\nСохранение результатов...")
    corr_df.to_csv(output_files["correlation_results"], index=False)
    pval_df.to_csv(output_files["p_value_results"], index=False)
    
    print(f"""
    Готово! Результаты сохранены:
    - Корреляции: {output_files['correlation_results']}
    - P-значения: {output_files['p_value_results']}
    """)

if __name__ == "__main__":
    main()

Загрузка и обработка данных...

Обработка LFC данных:
 - Начальное количество строк: 919
 - Найдено дубликатов ModelID: 0
 - Удалено строк: 0
 - Осталось уникальных ModelID: 919

Фильтрация по типам рака:
 - Общее число линий после фильтрации: 747
 - Учтённые типы рака (26):
primary_disease_renamed
Lung Cancer                   111
Glioma                         49
Melanoma                       48
Colon/Colorectal Cancer        46
Pancreatic Cancer              42
Ovarian Cancer                 41
Head and Neck Cancer           38
Breast Cancer                  33
Endometrial/Uterine Cancer     31
Bladder Cancer                 29
Gastric Cancer                 28
Esophageal Cancer              28
Bile Duct Cancer               27
Lymphoma                       21
Kidney Cancer                  21
Liver Cancer                   20
AML                            18
Ewing Sarcoma                  17
Neuroblastoma                  17
Rhabdoid ATRT Cancer           15
Cervical Cancer     